# Library

In [1]:
import os
import duckdb
import boto3
import pandas as pd
import psycopg2
from dotenv import load_dotenv
import json
load_dotenv()

True

# Performance Evaluation

## Postgres

### RDS Connection

In [ ]:
# RDS Crendtials
RDS_HOST = os.environ.get('RDS_HOST')
RDS_PORT = os.environ.get('RDS_PORT')
RDS_DATABASE = os.environ.get('RDS_DATABASE')
RDS_USER = os.environ.get('RDS_USER')
RDS_PASSWORD = os.environ.get('RDS_PASSWORD')
# RDS Postgres Connection
try:
    rds_conn = psycopg2.connect(
        host=RDS_HOST,
        port=RDS_PORT,
        database=RDS_DATABASE,
        user=RDS_USER,
        password=RDS_PASSWORD,
        sslmode='require'
    )
except Exception as e:
    print(f"Error connecting to RDS: {e}")
    raise

# RDS Postgres Cursor
rds_cursor = rds_conn.cursor()

In [ ]:
# Check available tables in the connected database
try:
    rds_cursor.execute("""
        SELECT table_schema, table_name 
        FROM information_schema.tables 
        WHERE table_type = 'BASE TABLE' AND table_schema NOT IN ('pg_catalog', 'information_schema')
        ORDER BY table_schema, table_name
    """)
    tables = rds_cursor.fetchall()
    for schema, table in tables:
        print(f"{schema}.{table}")
except Exception as e:
    print(f"Error fetching tables: {e}")
    raise

public.raw_ohlcv
public.silver_13d
public.silver_21d
public.silver_34d
public.silver_3d
public.silver_5d
public.silver_8d
public.symbol_metadata


### Resample

In [ ]:
import time

start_time = time.time()
try:
    rds_cursor.execute(sql)
    postgres_resampled_data = rds_cursor.fetchall()
    end_time = time.time()
    print(f"Resampling for AMD took {end_time - start_time:.4f} seconds")
    postgres_resampled_data
    postgres_resampling_time = end_time - start_time
except Exception as e:
    print(f"Error resampling data: {e}")
    raise

Resampling for AMD took 2.8069 seconds


### Query Testing

In [ ]:
import time
# Resampling with Postgres for only symbol 'AMD' and timing the processing
sql = f"""
    WITH numbered AS (
        SELECT
            symbol,
            DATE(timestamp) as date,
            open as open,
            high as high,
            low as low,
            close as close,
            volume,
            ROW_NUMBER() OVER (PARTITION BY symbol ORDER BY DATE(timestamp)) AS rn
        FROM raw_ohlcv
        WHERE interval = '1d'
            AND symbol in ('AMD')
    ),
    grp AS (
        SELECT
            symbol,
            date,
            open,
            high,
            low,
            close,
            volume,
            (rn - 1) / 3 AS grp_id
        FROM numbered
    ),
    aggregated AS (
        SELECT
            symbol,
            grp_id,
            MIN(date) AS start_date,
            MAX(high) AS high,
            MIN(low) AS low,
            SUM(volume) AS volume,
            (array_agg(open ORDER BY date))[1] AS open,
            (array_agg(close ORDER BY date DESC))[1] AS close
        FROM grp
        GROUP BY symbol, grp_id
        HAVING COUNT(*) = 3
    )
    SELECT
        symbol,
        start_date AS date,
        open,
        high,
        low,
        close,
        volume
    FROM aggregated
    ORDER BY symbol, date
"""

start_time = time.time()
try:
    rds_cursor.execute(
        '''
        SELECT ohlcv.* 
        FROM raw_ohlcv ohlcv
        JOIN symbol_metadata meta ON ohlcv.symbol = meta.symbol
        WHERE meta.industry = 'SEMICONDUCTORS & RELATED DEVICES'
            AND ohlcv.timestamp >= NOW() - INTERVAL '5 years'
        ORDER BY ohlcv.symbol, ohlcv.timestamp;
        '''
        )
    
    postgres_resampled_data = rds_cursor.fetchall()
    end_time = time.time()
    print(f"Fetching symbol from Semiconductor industry took in past 5 years took {end_time - start_time:.4f} seconds")
    postgres_resampled_data
except Exception as e:
    print(f"Error resampling data: {e}")
    raise

Fetching symbol from Semiconductor industry took in past 5 years took 48.1419 seconds


## DuckDB

### S3 Connection


In [2]:
import os   

conn = duckdb.connect()

In [3]:
# Test resampling logic with your migrated data using read_parquet
try:
    conn.execute("SET s3_region='ca-west-1'")
    conn.execute(f"SET s3_access_key_id='{os.environ.get('AWS_ACCESS_KEY_ID')}'")
    conn.execute(f"SET s3_secret_access_key='{os.environ.get('AWS_SECRET_ACCESS_KEY')}'")


    # Now create the view as before
    conn.execute("""
        CREATE OR REPLACE VIEW s3_ohlcv AS
        SELECT * FROM read_parquet('s3://dev-condvest-datalake/public/raw_ohlcv/*.parquet');
    """)
    
    # Check the data structure first
    result = conn.execute("""
        SELECT COUNT(*) as total_rows, 
            MIN(timestamp_1) as min_time, 
            MAX(timestamp_1) as max_time,
            COUNT(DISTINCT symbol) as unique_symbols
        FROM s3_ohlcv;
    """).fetchall()
    
    print("Data summary:")
    for row in result:
        print(f"Total rows: {row[0]}")
        print(f"Time range: {row[1]} to {row[2]}")
        print(f"Unique symbols: {row[3]}")
    
    
except Exception as e:
    print(f"❌ Error in resampling: {e}\n")
    print("Tip: Make sure you have set the correct AWS S3 region using DuckDB's SET s3_region statement.")


Data summary:
Total rows: 22609541
Time range: 1962-01-01 22:00:00-07:00 to 2025-10-02 22:00:00-06:00
Unique symbols: 5350


### Resample Processing

In [7]:
# Resampling with DuckDB for selected symbols, using s3_ohlcv view from S3 parquet data
sql = """
    WITH numbered AS (
        SELECT
            symbol,
            CAST(timestamp_1 AS DATE) AS date,
            open,
            high,
            low,
            close,
            volume,
            ROW_NUMBER() OVER (PARTITION BY symbol ORDER BY CAST(timestamp_1 AS DATE)) AS rn
        FROM s3_ohlcv
        WHERE interval = '1d'
        ORDER BY symbol, date
    ),
    grp AS (
        SELECT
            symbol,
            date,
            open,
            high,
            low,
            close,
            volume,
            CAST(FLOOR((rn - 1) / 3) AS INTEGER) AS grp_id
        FROM numbered
    ),
    aggregated AS (
        SELECT
            symbol,
            grp_id,
            MIN(date) AS start_date,
            MAX(high) AS high,
            MIN(low) AS low,
            SUM(volume) AS volume,
            FIRST(open ORDER BY date) AS open,
            FIRST(close ORDER BY date DESC) AS close
        FROM grp
        GROUP BY symbol, grp_id
        HAVING COUNT(*) = 3
    )
    SELECT
        symbol,
        start_date AS date,
        open,
        high,
        low,
        close,
        volume
    FROM aggregated
    ORDER BY symbol, date
"""

In [6]:
# Performance Comparison: DuckDB vs PostgreSQL
import time
print("=" * 50)

# Test DuckDB performance with S3 data
print("\n📊 DuckDB with S3 Data:")
start_time = time.time()

try:
    # Test resampling performance
    duckdb_result = conn.execute(sql).fetchdf()
    
    duckdb_time = time.time() - start_time
    print(f"✅ DuckDB resampling took: {duckdb_time:.4f} seconds")
    print(f"✅ DuckDB processed {len(duckdb_result)} rows")
    print(f"✅ DuckDB data shape: {duckdb_result.shape}")
    
    # Compare with PostgreSQL (from earlier cells)
    print(f"\n📊 PostgreSQL (from earlier test):")
    print(f"✅ PostgreSQL resampling took: {postgres_resampling_time:.4f} seconds")
    print(f"✅ PostgreSQL processed similar data")

    print(f"\n🏆 Performance Improvement:")
    if 'duckdb_time' in locals():
        improvement = postgres_resampling_time / duckdb_time
        print(f"✅ DuckDB is {improvement:.1f}x faster than PostgreSQL!")
        print(f"✅ Time saved: {postgres_resampling_time - duckdb_time:.4f} seconds")
except Exception as e:
    print(f"❌ DuckDB error: {e}")



📊 DuckDB with S3 Data:
✅ DuckDB resampling took: 47.1559 seconds
✅ DuckDB processed 7534718 rows
✅ DuckDB data shape: (7534718, 7)

📊 PostgreSQL (from earlier test):
❌ DuckDB error: name 'postgres_resampling_time' is not defined


In [9]:
duckdb_result[duckdb_result['symbol'] == 'AMD']

,symbol,date,open,high,low,close,volume
345400,AMD,1980-03-16,3.13,3.30,2.94,3.04,1242000.0
345401,AMD,1980-03-19,3.04,3.06,2.64,2.67,727200.0
345402,AMD,1980-03-24,2.67,2.75,2.24,2.38,2241600.0
345403,AMD,1980-03-27,2.46,2.77,2.46,2.74,1588800.0
345404,AMD,1980-04-01,2.74,2.74,2.53,2.54,865200.0
...,...,...,...,...,...,...,...
349222,AMD,2025-09-11,157.00,162.30,154.92,160.46,106988000.0
349223,AMD,2025-09-16,159.28,161.63,149.85,157.39,181821400.0
349224,AMD,2025-09-21,157.42,165.10,157.42,160.88,124417900.0
349225,AMD,2025-09-24,157.14,164.30,154.78,161.36,106955600.0


### Plot the Reampled Result

In [7]:
# Plot the resampled data for last 100 days
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Candle plot for last 100 days
filtered_data = duckdb_result[(duckdb_result['symbol'] == 'AMD') & (duckdb_result['date'] >= (duckdb_result['date'].max() - pd.Timedelta(days=365)))]

fig = make_subplots(
    rows=2,
    cols=1,
    shared_xaxes=True,
    vertical_spacing=0.02,
    row_heights=[0.7, 0.3]  # Candlestick gets more height than volume
)

# Add candlestick trace to row=1, col=1
fig.add_trace(
    go.Candlestick(
        x=filtered_data['date'],
        open=filtered_data['open'],
        high=filtered_data['high'],
        low=filtered_data['low'],
        close=filtered_data['close'],
        name='Candlestick'
    ),
    row=1,
    col=1
)

# Add volume trace to row=2, col=1
fig.add_trace(
    go.Bar(
        x=filtered_data['date'],
        y=filtered_data['volume'],
        name='Volume',
        marker_color='lightblue'
    ),
    row=2,
    col=1
)

# Update y-axes titles individually
fig.update_yaxes(title_text='Price', row=1, col=1)
fig.update_yaxes(title_text='Volume', row=2, col=1)

# General layout updates
fig.update_layout(
    title='Candlestick and Volume Chart for Last 365 Days',
    xaxis=dict(title='Date', rangeslider_visible=False),
    height=700,
    showlegend=False
)

# Show the plot
fig.show()

## Polars

In [7]:
# import polars as pl

# # Read from RDS
# uri = f"postgresql://{RDS_USER}:{RDS_PASSWORD}@{RDS_HOST}:{RDS_PORT}/{RDS_DATABASE}?sslmode=require"
# query = "SELECT * FROM raw_ohlcv"
# pl.read_database_uri(query, uri)  